In [1]:
from utils import *
import c3.libraries.algorithms as algorithms
from SingleQubitExperiment import SingleQubitExperiment

2021-09-02 18:44:51.065804: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-02 18:44:51.065857: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:

def test_fidelity(propagators: dict, instructions: dict, index, dims,
                  qubit: chip.Qubit, generator: Generator, drive: chip.Drive, n_eval=-1):
    numPeaks = 2
    # find energies of the qubit
    energies = qubit.get_Hamiltonian().numpy().diagonal().real
    vec1 = np.array([energies[3] - energies[2], energies[1] - energies[0]]) / (2 * np.pi)

    infids = []
    for gate, propagator in propagators.items():
        # create signal and find peaks
        instr = instructions[gate]
        signal = generator.generate_signals(instr)[drive.name]
        peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), numPeaks)

        # add frequency offsets to infidelity
        #infid = np.linalg.norm(vec1 - peakFrequencies)
        infid1 = np.abs(vec1[0] - peakFrequencies[0]) / peakFrequencies[0]
        infid1 += np.abs(vec1[1] - peakFrequencies[1]) / peakFrequencies[1]

        # add peak height difference to infidelity
        if peakFrequencies[1] > peakFrequencies[0]:
            a = peakValues[1]
            b = np.sqrt(2) * peakValues[0]
        else:
            a = peakValues[0]
            b = np.sqrt(2) * peakValues[1]
        infid2 = np.abs(a - b) / np.maximum(a, b)
        infid = infid1 * (1 + infid2)
        print("peaks: ", np.sort(peakFrequencies), " => ", vec1, "; infidelity: ", infid1, infid2, infid)
        infids.append(infid)
    return tf.reduce_mean(infids)


def runExperiment(
        t_final: float,
        freq: float,
        anharm: float,
        carrier_freq: float,
        envelope: pulse.Envelope,
        optimisable_params: dict,
        algorithm: Callable,
        algorithm_options: {},
        file_suffix: str,
        directory: str = None,
        num_gates=1
) -> dict:
    exper = SingleQubitExperiment(file_suffix=file_suffix, directory=directory)
    exper.prepare(t_final, freq, anharm, carrier_freq, envelope)
    return exper.optimise(optimisable_params, algorithm, algorithm_options)

In [5]:
freq = 5e9
anharm = 330e6
t_final = 10e-9
sigma = t_final / 15
sigma2 = sigma
relative_amp = np.sqrt(2)
carrier_freq = 4.7e9
optimisable_params = {
    "gauss": ["amp", "sigma",
              #"xy_angle", "freq_offset",  # for single gaussian envelope
              #"t_final", "t_bin_start", "t_bin_end",
              "sigma2", "relative_amp",  # for double-gaussian envelope
              "delta",  # for DRAG
              #"inphase",
              ],
    "carrier": [
        "freq",
        "framechange"
    ]
}

#amp = 2.9898805422620756 #V
#sigma = 7.914863107804569e-10 #s
#sigma2 = 6.955276846125239e-10 #s
#relative_amp = 1.8131542761817843 #
#delta = 0.004339356027326602 #
#carrier_freq = 4655811329.747005 #Hz 2pi
#framechange = 2.55339714228929 #rad

#for idx, delta in enumerate(np.linspace(-1, 5, 50)):
envelope = createDoubleGaussianPulse(t_final, sigma, sigma2, relative_amp)
paramsResult = runExperiment(t_final, freq, anharm, carrier_freq,
                             envelope=envelope,
                             optimisable_params=optimisable_params,
                             algorithm=algorithms.cmaes,
                             algorithm_options={"stop_at_convergence": 30},
                             #file_suffix=f"{idx}_{delta}",
                             file_suffix="",
                             directory="./output/test",
                             num_gates=1
                             )
print(paramsResult)

energies:  [5000000000.0, 4670000000.000003, 4339999999.999994, 4010000000.0000033]
saving plot in ./output/test/signal_before_.png
peaks:  [3.30165083e+09 4.40220110e+09 5.10255128e+09 6.20310155e+09]
saving plot in ./output/test/populations_before_.png
C3:STATUS:Saving as: /home/user/c3/output/test/log__10.00/c1_unitary_infid_set_cmaes/2021_09_02_T_18_46_22/open_loop.log
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=670059, Thu Sep  2 18:46:22 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 7.244079174581552e-01 1.0e+00 9.65e-02  9e-02  1e-01 0:06.1
    2     18 6.027187497652191e-01 1.2e+00 9.91e-02  9e-02  1e-01 0:12.8
    3     27 4.898990690625926e-01 1.3e+00 9.50e-02  8e-02  1e-01 0:19.7
before:
 lower-X[0]-d1-gauss-amp               : 3.000 V 
lower-X[0]-d1-gauss-sigma             : 666.667 ps 
lower-X[0]-d1-gauss-sigma2            : 666.667 ps 
lower-X[0]-d1-gauss-relative_amp      : 1.414  
lower-X[0]-d1-gauss-delta        